# 西安项目数据预处理

## 预设

In [1]:
import sys
import os
import openpyxl
import csv
import re
import pandas as pd
import jieba
import jieba.posseg as psg
import matplotlib.pyplot as plt
import numpy as np
import cn2an
import functools
from loguru import logger
logger.remove()
handler_id = logger.add(sys.stderr, level="DEBUG")
# 支持中文
plt.rcParams['font.sans-serif'] = ['SimSong-Regular'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False # 用来正常显示负号
# 显示清晰
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### 全局参数

In [8]:
ExtraDictionaryPath = 'set/dict.txt'
StopWordPath = 'set/stopwords.txt'
KeywordTopNumber = 10
minLengthOfWord = 2
DictionaryFilePath = 'data/西安项目编码表1.pkl'
DataFilePath = 'data/5西安项目分类后二分词.pkl'

### 导入数据

In [9]:
dictionary_data = pd.read_pickle(DictionaryFilePath)
data = pd.read_pickle(DataFilePath)

## 函数工具

### 字符串工具

In [5]:
#分词工具
def chinese_word_cut(my_text, min_length_of_word = 2, flag_list = ['n','nz','vn']):
    jieba.load_userdict(ExtraDictionaryPath)
    jieba.initialize()
    try:
        stopword_list_unprocessed = open(StopWordPath,encoding ='utf-8')
    except:
        stopword_list_unprocessed = []
        print("error in stop_file")
    stop_word_list = []
    for line in stopword_list_unprocessed:
        line = re.sub(u'\n|\\r', '', line)
        stop_word_list.append(line)

    #jieba分词
    word_list = []
    seg_list = psg.cut(my_text)
    for seg_word in seg_list:
        # word = re.sub(u'[^\u4e00-\u9fa5]','',seg_word.word)
        word = seg_word.word
        is_unfit = False
        for stop_word in stop_word_list:
            if stop_word == word or len(word) <  min_length_of_word:     #this word is stopword
                    is_unfit = True
                    break
        if (not is_unfit) and seg_word.flag in flag_list:
            word_list.append(word)      
    return (" ").join(word_list)

In [6]:
# 安全分词
def safe_cut_words(words_str, seg=' '):
    if not seg:
        return
    words_str = re.sub(f'[{seg}]+','|', words_str).strip('|')
    if not words_str:
        return 
    return words_str.split('|')

In [7]:
# 选取高频词（在词集中）
def select_high_frequency_words(words_str, seg=' ', top_num = 10):
    word_list = safe_cut_words(words_str, seg)
    if not word_list:
        return '', 0
    word_frequency_series = pd.Series(word_list, name="count")
    word_frequency_rank = pd.DataFrame()
    word_frequency_rank['count'] = word_frequency_series.value_counts()
    total_num = word_frequency_rank['count'].size
    if top_num > total_num:
        top_num = total_num - 1
    word_frequency_rank_top = word_frequency_rank[word_frequency_rank['count']>=word_frequency_rank['count'][top_num - 1]]
    return ' '.join(word_frequency_rank_top['count'].index), word_frequency_rank['count'].size

In [8]:
# 列表重合检测
def list_detection(list1, list2) -> bool:
    return frozenset(list1).intersection(list2)

In [9]:
# 词典检测
def dictionary_detection(words_str, dic_df, dictionary_column_name, seg = ' '):
    word_list = safe_cut_words(words_str, seg)
    if not word_list:
        return 0, ''
    
    keywords_list = dic_df[dic_df[dictionary_column_name].notna()][dictionary_column_name].to_list()
    
    coincident_keyword_set = list_detection(word_list, keywords_list)
    
    if len(coincident_keyword_set) > 0:
        return 1, ' '.join(coincident_keyword_set)
    
    return 0, ''

### pandas工具

In [15]:
# pandas一列生成多列的工具
def apply_and_concat(df, field, func, args, column_names):
    return pd.concat((
        df,
        df[field].apply(
            lambda cell: pd.Series(func(cell, *args), index=column_names))), axis=1)

In [10]:
# 获取分类
def get_classification(data_df, data_words_column_name, dic_df):
    for class_name,class_series in dic_df.iteritems():  
        data_df = apply_and_concat(data_df, data_words_column_name, dictionary_detection, args=(dic_df, class_name), column_names=[class_name, class_name+'选中词'])
    return data_df

In [11]:
# 选取高频词（在数据集中）
def get_high_frequency_words_series(row, words_column_name, top_num = 10):
    row[words_column_name+'高频词'], row[words_column_name+'高频词数'] = select_high_frequency_words(row[words_column_name], top_num)
    return row

### 新保存工具

In [4]:
def get_new_path(old_path, new_tag):
    path_list = old_path.split('.')
    path_list[0] += new_tag
    return '.'.join(path_list)

## 数据处理

### 分词操作

In [ ]:
data['分词'] = data['内容'].map(chinese_word_cut, args=(minLengthOfWord, ))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3b/vr_bzyy109l8shrht3p8h5700000gn/T/jieba.cache
Loading model cost 1.113 seconds.
Prefix dict has been built successfully.


,内容,发布时间,点赞数,收藏数,转发数,user_tag,平台,评论数,分词
0,西安旅游攻略吧，小伙伴年底要来西安玩一周！😂😂😂太高兴了。但是你要问一个西安人，西安有啥玩的...,2021-11-21 08:14:20,64,144,18,https://www.douban.com/people/Ch95134/,豆瓣,0,小伙伴 肉夹馍 葫芦头 猪大肠 八宝粥 本地人 历史博物馆 博物馆 博物馆 博物馆 专业人士...
1,西安回民街到底该不该去，是不是专门坑游客的地方五一到了，很多人来西安玩会有个疑惑回民街到底该...,2021-04-24 15:58:39,186,442,38,https://www.douban.com/people/nnnpppccc/,豆瓣,0,一条街 感兴趣 互联网 一条街 小土豆 臭豆腐 本地人 本地人 聚居区 马二酸汤 本地人 粉...
2,西安怕是不能再来了~我现在对我的西安朋友充满了怨愤。因为他们要是早一点告诉我西安这么好吃，我...,2019-09-16 18:42:50,758,2782,613,https://www.douban.com/people/ChannaDisco/,豆瓣,0,欧洲人 素食主义者 盐焗鸡 葱油鸡 辣椒面 男朋友 篇文章 豆腐脑 豆腐脑 利口酒 前男友 ...
3,西安是个很好玩的城市，景点和骗子都很好玩。首先想说的是，作为十三朝古都，西安是一个非常好的城...,2017-01-05 13:16:47,10789,11026,2816,https://www.douban.com/people/xiangyi816/,豆瓣,0,篇文章 一叶障目 篇文章 兵马俑 火车站 公交车 兵马俑 火车站 兵马俑 兵马俑 兵马俑 公...
4,博物馆游记｜西安（三）第四站·秦始皇帝陵博物院（秦始皇兵马俑博物馆）秦始皇兵马俑，号称“世界...,2021-06-26 20:25:30,10,6,1,https://www.douban.com/people/qubing/,豆瓣,0,博物馆 博物院 兵马俑博物馆 兵马俑 遗宝展 猴年马月 时间段 遗宝展 兵马俑博物馆 发车场...
...,...,...,...,...,...,...,...,...,...
20207,高新、软件新城、沣东的交界,2021-05-06 10:48,0,0,0,https://www.zhihu.com/people/li-xiang-yu-54-3,知乎,0,
20208,西安的环路真的是，南三环平均只敢开60，一路红绿灯加大井盖泥土的，一堵就是四五百米，东西三环...,2021-04-15 10:23,3,0,0,https://www.zhihu.com/people/jiu-mei-zai-na-li,知乎,0,三环路
20209,软件新城上班，每天经过西三环，鱼化寨。附近交通比较乱，三环边上的绿植总是一层灰，没有啥自行车...,2021-03-06 07:25,3,0,0,https://www.zhihu.com/people/science-2,知乎,0,自行车道 鱼化寨
20210,比东三环差多了，都什么年代了还在居民集中区建330kv超高压线，拿着十年年的环评报告施工，十...,2021-11-24 17:55,2,0,0,https://www.zhihu.com/people/simplehappy-57,知乎,1,集中区 高压线 居民楼 高压线 高压线


### 选取高频词

In [ ]:
data = data.apply(get_high_frequency_words_series, args=('分词', KeywordTopNumber),axis=1)

### 高频词第二种操作(结果好像这种更快)

In [ ]:
data = apply_and_concat(data, '分词', select_high_frequency_words, args=(KeywordTopNumber,), column_names=['分词高频词', '分词高频词数'])

### 构建‘框架词典’的dict

In [13]:
dictionary_comparison_table = pd.DataFrame()
for i,v in dictionary_data['二级框架类型'].items():
    dictionary_comparison_table[v] = pd.Series(safe_cut_words(dictionary_data.loc[i,'框架关键词'], '、'))

### 检测框架

In [14]:
data = get_classification(data, '分词高频词', dictionary_comparison_table)

In [ ]:
dictionary_detection(['gdp',
 '房地产',
 '开发区',
 '开发商',
 '企事业',],dictionary_comparison_table, '经济发展')

(1, 'gdp 房地产 开发商 企事业 开发区')

## 结果展示&保存

### 结果展示

In [16]:
df = data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71051 entries, 0 to 71050
Data columns (total 43 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   内容         71051 non-null  object
 1   发布时间       71051 non-null  object
 2   点赞数        71051 non-null  int64 
 3   收藏数        71051 non-null  int64 
 4   转发数        71051 non-null  int64 
 5   user_tag   71051 non-null  object
 6   平台         71051 non-null  object
 7   评论数        71051 non-null  int64 
 8   分词         71051 non-null  object
 9   分词高频词      71051 non-null  object
 10  分词高频词数     71051 non-null  int64 
 11  政治环境       71051 non-null  int64 
 12  政治环境选中词    71051 non-null  object
 13  经济发展       71051 non-null  int64 
 14  经济发展选中词    71051 non-null  object
 15  生态建设       71051 non-null  int64 
 16  生态建设选中词    71051 non-null  object
 17  文化风俗       71051 non-null  int64 
 18  文化风俗选中词    71051 non-null  object
 19  重大事件       71051 non-null  int64 
 20  重大事件选中词    71051 non-null  o

In [ ]:
df.index

RangeIndex(start=0, stop=86, step=1)

In [ ]:
df.shape

(86, 16)

In [ ]:
df[df.isnull().T.any()]

,一级框架类型,二级框架类型,框架说明,框架关键词,编号,总类型


In [ ]:
df[df['编号']=='nan']

,一级框架类型,二级框架类型,框架说明,框架关键词,编号,总类型


In [ ]:
display(df)

,政治环境,经济发展,生态建设,文化风俗,重大事件,司法治安,医疗卫生,基础设施,科教文艺,休闲娱乐,社会民生,市民样貌,积极判断框架,消极判断框架,事件框架,细节框架
0,办事处,gdp,银杏树,阿房宫,世博园,110,高风险,出租车,211,app,tips,安全感,安全感,高风险,110,App
1,宝鸡市,房地产,大自然,白鹿原,意大利,当事人,流行病,地铁站,985,biangbiang,办公室,打招呼,标志性,流行病,211,biangbiang
2,标志性,开发区,红豆杉,兵马俑,交流会,派出所,传染源,高峰期,本科生,ktv,毕业生,当地人,车水马龙,官僚主义,985,tips
3,代表性,开发商,NaN,博物馆,洽谈会,被告人,心脏病,高速公路,分数线,skp,成年人,名副其实,大都市,乱七八糟,gdp,爱好者
4,对外开放,企事业,NaN,博物院,奥运会,公安部,hpv,火车票,机器人,爱好者,打电话,幸福感,代表性,买不起,skp,本科生
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,联合会,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,买不起,NaN,NaN,NaN,一整天,志愿者
82,内蒙古,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,年收入,NaN,NaN,NaN,医学院,中小学
83,乌鲁木齐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,年终奖,NaN,NaN,NaN,义务教育,自行车
84,专家组,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,上下班,NaN,NaN,NaN,意大利,总经理


### 数据保存

In [5]:
new_path = get_new_path(DataFilePath, '二分词')

In [ ]:
df.to_pickle(new_path)

## palyground


In [1]:
data = data[:10]
data = get_classification(data, '分词高频词', dictionary_comparison_table)

NameError: name 'data' is not defined

In [17]:
display(data)

,内容,发布时间,点赞数,收藏数,转发数,user_tag,平台,评论数,分词,分词高频词,...,市民样貌,市民样貌选中词,积极判断框架,积极判断框架选中词,消极判断框架,消极判断框架选中词,事件框架,事件框架选中词,细节框架,细节框架选中词
0,西安旅游攻略吧，小伙伴年底要来西安玩一周！😂😂😂太高兴了。但是你要问一个西安人，西安有啥玩的...,2021-11-21 08:14:20,64,144,18,https://www.douban.com/people/Ch95134/,豆瓣,0,小伙伴 肉夹馍 葫芦头 猪大肠 八宝粥 本地人 历史博物馆 博物馆 博物馆 博物馆 专业人士...,博物馆 本地人 未央区 文公庙 电视塔 森林公园 肉夹馍 汽车站 客运站 兵马俑 八宝粥 咖...,...,0,,0,,0,,1,博物馆 客运站 火车站 兵马俑 动物园,1,不倒翁 小伙伴 羽绒服
1,西安回民街到底该不该去，是不是专门坑游客的地方五一到了，很多人来西安玩会有个疑惑回民街到底该...,2021-04-24 15:58:39,186,442,38,https://www.douban.com/people/nnnpppccc/,豆瓣,0,一条街 感兴趣 互联网 一条街 小土豆 臭豆腐 本地人 本地人 聚居区 马二酸汤 本地人 粉...,本地人 粉蒸肉 一条街 小土豆 聚居区 美食节 感兴趣 马二酸汤 外地人 基本上 互联网 臭...,...,0,,0,,0,,0,,1,互联网 外地人
2,西安怕是不能再来了~我现在对我的西安朋友充满了怨愤。因为他们要是早一点告诉我西安这么好吃，我...,2019-09-16 18:42:50,758,2782,613,https://www.douban.com/people/ChannaDisco/,豆瓣,0,欧洲人 素食主义者 盐焗鸡 葱油鸡 辣椒面 男朋友 篇文章 豆腐脑 豆腐脑 利口酒 前男友 ...,男朋友 豆腐脑 朋友圈 消食片 盐焗鸡 当地人 素食主义者 本地人 欧洲人 前男友 葱油鸡 ...,...,1,当地人,0,,0,,0,,1,当地人 男朋友
3,西安是个很好玩的城市，景点和骗子都很好玩。首先想说的是，作为十三朝古都，西安是一个非常好的城...,2017-01-05 13:16:47,10789,11026,2816,https://www.douban.com/people/xiangyi816/,豆瓣,0,篇文章 一叶障目 篇文章 兵马俑 火车站 公交车 兵马俑 火车站 兵马俑 兵马俑 兵马俑 公...,兵马俑 博物院 公交车 停车场 火车站 篇文章 全名是 工作人员 真糟糕 篮球场 本地人 光...,...,0,,0,,0,,1,兵马俑 博物院 停车场 火车站,1,停车位
4,博物馆游记｜西安（三）第四站·秦始皇帝陵博物院（秦始皇兵马俑博物馆）秦始皇兵马俑，号称“世界...,2021-06-26 20:25:30,10,6,1,https://www.douban.com/people/qubing/,豆瓣,0,博物馆 博物院 兵马俑博物馆 兵马俑 遗宝展 猴年马月 时间段 遗宝展 兵马俑博物馆 发车场...,讲解员 兵马俑 博物馆 遗宝展 兵马俑博物馆 商业街 停车场 发车场 锡天香 猴年马月 大巴...,...,0,,0,,0,,1,兵马俑 博物院 博物馆 停车场,1,自行车 小朋友 讲解员
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71046,高新、软件新城、沣东的交界,2021-05-06 10:48,0,0,0,https://www.zhihu.com/people/li-xiang-yu-54-3,知乎,0,,,...,0,,0,,0,,0,,0,
71047,西安的环路真的是，南三环平均只敢开60，一路红绿灯加大井盖泥土的，一堵就是四五百米，东西三环...,2021-04-15 10:23,3,0,0,https://www.zhihu.com/people/jiu-mei-zai-na-li,知乎,0,三环路,三环路,...,0,,0,,0,,0,,0,
71048,软件新城上班，每天经过西三环，鱼化寨。附近交通比较乱，三环边上的绿植总是一层灰，没有啥自行车...,2021-03-06 07:25,3,0,0,https://www.zhihu.com/people/science-2,知乎,0,自行车道 鱼化寨,自行车道 鱼化寨,...,0,,0,,0,,0,,0,
71049,比东三环差多了，都什么年代了还在居民集中区建330kv超高压线，拿着十年年的环评报告施工，十...,2021-11-24 17:55,2,0,0,https://www.zhihu.com/people/simplehappy-57,知乎,1,集中区 高压线 居民楼 高压线 高压线,高压线 居民楼 集中区,...,0,,0,,0,,0,,0,
